# CSCI451 Project: UNHCR ML Challenge
Jamie Hackney, Mihir Singh, Jake Gilbert

In [26]:
# imports
import numpy as np
import pandas as pd
# imports for models
from sklearn.linear_model import LinearRegression
from sklearn import tree 
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

# create models
LR = LinearRegression()
Svc = SVC()
Tree = tree.DecisionTreeClassifier()
NN = MLPClassifier()

# load the data
df = pd.read_csv('/Users/mihirsingh/Documents/Middlebury/CSCI451/UNHCR-ml-challenge/data/combined_data.csv')
df_imp = df.copy()

# encode data that is not numerical
encoder = LabelEncoder()
encoder.fit(df_imp['Region'])
df_imp['Region'] = encoder.transform(df_imp['Region'])
encoder.fit(df_imp['District'])
df_imp['District'] = encoder.transform(df_imp['District'])
encoder.fit(df_imp['Month'])
df_imp['Month'] = encoder.transform(df_imp['Month'])

# turn string numbers into floats
features = ['CDI','Month','Year','NDVI','Rainfall','Water Price',
            'Conflict Fatalities','Conflict Incidents','Cholera Deaths',
            'Cholera Cases','Malaria','Measles','Cost Min Basket',
            'Goat Price','Goat to Cereal','Maize Price','Rice Price',
            'Sorghum Price','Wage Price','Wage to Cereal', 'Departures','Arrivals']

for feature in features:
    df_imp[feature] = df_imp[feature].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)
    
# impute data
# imp = SimpleImputer(missing_values=np.nan, strategy='median')
# array_imp = imp.fit_transform(df_imp)
# df_imp = pd.DataFrame(array_imp, columns=df.columns)

imp = IterativeImputer(max_iter=10, random_state=0)
array_imp = imp.fit_transform(df_imp)
df_imp = pd.DataFrame(array_imp, columns=df.columns)   

df_imp.head()

/var/folders/fv/_v7myjl904983sbb39qbkzw40000gn/T/ipykernel_94000/1946157502.py:23: DtypeWarning: Columns (17,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/mihirsingh/Documents/Middlebury/CSCI451/UNHCR-ml-challenge/data/combined_data.csv')
/Users/mihirsingh/anaconda3/envs/ml-0451/lib/python3.9/site-packages/sklearn/impute/_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,Unnamed: 0,Region,District,CDI,Month,Year,NDVI,Rainfall,Water Price,Conflict Fatalities,...,Cost Min Basket,Goat Price,Goat to Cereal,Maize Price,Rice Price,Sorghum Price,Wage Price,Wage to Cereal,Departures,Arrivals
0,0.0,0.0,13.0,1.05,4.0,2018.0,0.168,4.342,4000.000000,0.0,...,1.090175e+06,2.977500e+05,57.000000,4000.000000,5200.000000,3200.000000,57400.000000,11.000000,6.000000,199.0
1,1.0,0.0,5.0,0.62,4.0,2018.0,0.132,6.065,8816.146932,0.0,...,3.221842e+06,1.034149e+06,96.505868,12674.026322,20302.198169,15778.178913,103823.463756,9.562667,415.870257,69.0
2,2.0,0.0,53.0,0.63,4.0,2018.0,0.047,7.915,15724.612735,0.0,...,1.674934e+06,3.000000e+05,50.000000,4729.703777,7000.000000,6000.000000,90000.000000,15.000000,253.000000,24.0
3,3.0,0.0,73.0,1.10,4.0,2018.0,0.052,8.121,6000.000000,0.0,...,1.160475e+06,4.100000e+05,59.000000,6000.000000,7000.000000,5500.000000,100000.000000,14.000000,-32.184029,68.0
4,4.0,17.0,41.0,0.78,4.0,2018.0,0.000,2.247,11000.000000,0.0,...,1.366665e+06,4.600000e+05,92.000000,6000.000000,5000.000000,5000.000000,70000.000000,14.000000,400.000000,663.0


In [27]:
df_imp.shape

(328856, 25)

In [28]:
# split data into training and testing data
columns = ['CDI','Month','Year','NDVI','Rainfall','Water Price',
            'Conflict Fatalities','Conflict Incidents','Cholera Deaths',
            'Cholera Cases','Malaria','Measles','Cost Min Basket',
            'Goat Price','Goat to Cereal','Maize Price','Rice Price',
            'Sorghum Price','Wage Price','Wage to Cereal', 'Departures']
X_train, X_test, y_train, y_test = train_test_split(df_imp[columns], df_imp['Arrivals'], test_size=0.15)
X_train.head()
X_train.shape


(279527, 21)

With the data processed, we can now build some models.

In [29]:
def has_negative(values):
    for value in values:
        if value < 0:
            return True
    return False


X_train.head()
list(y_train)

[445.0,
 261.0,
 981.0,
 179.0,
 1618.0,
 362.76844434440136,
 1904.0,
 259.0,
 -28.937619134783745,
 970.7860896289349,
 475.0,
 816.0,
 261.0,
 113.69490417093039,
 1876.7046327143908,
 551.4815684556961,
 188.0,
 1399.0,
 1107.5497242063284,
 21.0,
 45.0,
 19.0,
 103.0,
 1288.0,
 600.1866604909301,
 -1229.489833213389,
 -8705.387980788946,
 560.0,
 11.0,
 873.6945407539606,
 -376.55490873008966,
 22.0,
 503.0,
 -646.4085885584354,
 610.1106046959758,
 7507.0,
 28.0,
 178.17591500282288,
 64.0,
 13.0,
 1028.9121128246188,
 -520.4672205671668,
 322.0,
 2302.8895513266325,
 1456.10358043015,
 281.7081555277109,
 2360.0,
 -744.6501461341977,
 145.11791390925646,
 -219.58592873066664,
 153.0,
 29.0,
 -473.3956663310528,
 204.0,
 116.59632451832294,
 376.2793243974447,
 27.0,
 574.0899833962321,
 46.0,
 -250.28704019635916,
 29.0,
 -250.93429297953844,
 587.0,
 16.0,
 420.34892243891954,
 136.0,
 2490.0,
 694.5805999711156,
 -64.83229380100965,
 637.0,
 36.0,
 888.0,
 100.0,
 176.0,
 559.

In [30]:
# fitting models
LR.fit(X_train, list(y_train))

# predicting
LR_pred = LR.predict(X_train)
# evaluate
print("on training data: ", mean_squared_error(list(y_train), LR_pred))


on training data:  4622056.836219689
